In [71]:
from elasticsearch import Elasticsearch

# Singleton class, use variable es_handle
class EsHandle:
    def __init__(self):
        self._products_index = 'products'
        self._qa_index = 'qa'
        self._es = Elasticsearch()

    # Query all the indices that aren't system indices in the elasticsearch
    def get_useful_indices(self):
        filtered_idx = filter(lambda x : '.' not in x, [f['index'] for f in self._es.cat.indices(format='json')])
        indices_names = [idx for idx in filtered_idx]
        return indices_names

    # Get the product in the 'products' index by name
    def get_product_by_name(self, name):
        query_body = {
            "query": {
                "match": {
                    "product": name
                }
            }
        }
        possible_products = []
        query = self._es.search(index=self._products_index, body=query_body)['hits']['hits']
        
        for p in query:
            possible_products.append(p['_source'])
        
        return possible_products
    
    def get_qa_by_asin(self, asin, question):
        query_body = {
            "query": {
                "bool": {
                "must": [
                    {
                    "match": {
                        "asin": asin
                    }
                    },
                    {
                    "match": {
                        "question": question
                    }
                    }
                ]
                }
            }
            }

        possible_answers = []
        query = self._es.search(index=self._qa_index, body=query_body)['hits']['hits']
        
        for p in query:
            possible_answers.append(p['_source'])
        
        return possible_answers

    # Get the products names and id's from the 'products' index
    def get_products(self):
        result = []
        query = self._es.search(index=self._products_index)['hits']['hits']
        
        for p in query:
            p_id = p['_id']
            p_name = p['_source']['product']
            result.append((p_name, p_id))
        return result
    
    # Get specific product fact
    def get_product_fact(self, product_name, product_fact):
        products_jsons = self.get_product_by_name(product_name)
        possible_products = []

        for product in products_jsons:
            title = product.get("product", "TITLE")
            fact_json = product["product_info"]
            value = fact_json.get(product_fact, "FACT")
            possible_products.append({"Produto": title, product_fact: value})
        
        return possible_products

    def get_product_qa(self, product_name, question):
        products_jsons = self.get_product_by_name(product_name)
        if(len(products_jsons) < 1):
            return []

        product = products_jsons[0]
        title = product["product"]
        add_info_json = product["additional_info"]
        asin = add_info_json["ASIN"]
        
        value = self.get_qa_by_asin(asin, question)
        
        return {"Produto": title, "qa": value}

    def get_product_price(self, product_name):
        products_jsons = self.get_product_by_name(product_name)
        possible_products = []

        for product in products_jsons:
            title = product.get("product", "TITLE")
            price = product.get("price", "PRICE")
            possible_products.append({"Produto": title, "price": price})
        
        return possible_products

    def get_product_url(self, product_name):
        products_json = self.get_product_by_name(product_name)
        possible_products = [{"Produto": prod.get("product", "TITLE"), "URL": f"www.amazon.com.br/gp/product/{prod['additional_info']['ASIN']}"} for prod in products_json]
        return possible_products 


# Singleton implementation
es_handle = EsHandle()

In [3]:
possible_products = es_handle.get_product_by_name("iPhone")
possible_products

[{'title': "Celular Apple iPhone 11 64gb / Tela 6.1'' / 12MP / iOS 13 | Amazon.com.br",
  'product': "Celular Apple iPhone 11 64gb / Tela 6.1'' / 12MP / iOS 13",
  'price': 'R$4.599,90',
  'product_info': {'Sistema operacional': 'Ios',
   'RAM': '64 GB',
   'Capacidade de armazenamento da memória': '64 GB',
   'Capacidade de armazenamento digital': '64 GB',
   'Pilha(s) ou bateria(s):': '1 Íon de lítio baterias ou pilhas necessárias (inclusas).',
   'Número do modelo': 'MWLU2QL/A',
   'Tamanho de tela vertical': '6.1 Polegadas',
   'Tecnologia da tela': 'LCD',
   'Outras características de tela': 'Wireless',
   'Formato': 'Smartphone',
   'Cor': 'Branco',
   'Número de unidades': '1',
   'Modelos compatíveis': 'IPhone',
   'Peso do produto': '470 g',
   'Dimensões do produto': '17 x 9 x 5 cm; 470 g',
   'Marca': 'Apple',
   'Funciona a bateria ou pilha?': 'Sim',
   'EAN': '0190199221314, 0190199221505'},
  'additional_info': {'Dimensões do pacote': '17 x 9.4 x 5.2 centímetros',
   'ASI

In [72]:
es_handle.get_product_qa("Carro", "vem com carregador")

[]